In [64]:
import pandas as pd
import numpy as np

def load_data(file, sheet_name):
    dataframe=pd.read_excel(file, sheet_name=sheet_name, index=False)
    dataframe=dataframe.fillna(0)
    dataframe['today']=(dataframe.sum(axis=1)-dataframe.min_req)-dataframe.priority
    dataframe['needed']=np.where(dataframe.min_req-dataframe.today<0, 0, dataframe.min_req-dataframe.today)
    total=dataframe.apply(np.sum)
    total.svc='total'
    cols=dataframe.columns.tolist()
    dataframe=dataframe[cols]
    dataframe=dataframe.append(pd.DataFrame(total.values, index=total.keys()).T, ignore_index=True)
    return dataframe

df_curr=load_data('staff.xlsx', 'current')
df_min=load_data('staff.xlsx', 'min')

In [65]:
def get_min(col, idx):
    #for x in df_min.columns:
     #   for y in range(len(df_min)):
    n=df_min[col].values[idx]
    return n

get_min('equip_op', 0)

5.0

In [66]:
df_curr

,svc,priority,cdl,drivers,laborers,admin,cashier,equip_op,supers,emer_drivers,min_req,today,needed
0,qrl,1,6,0,12,0,2,0,3,0,12,23,0
1,payroll,1,0,0,0,10,0,0,0,0,9,10,0
2,routine,2,46,0,144,0,0,0,16,0,207,206,1
3,nwts,2,3,0,20,0,1,5,20,0,13,49,0
4,office,3,0,0,0,18,0,0,0,0,2,18,0
5,conv_ctr,3,0,0,0,0,0,0,0,0,14,0,14
6,ds_da,3,9,6,33,0,0,9,2,0,41,59,0
7,mss,3,15,0,0,0,0,0,0,0,0,15,0
8,prop,4,10,0,33,0,0,10,0,0,0,53,0
9,spec,4,6,14,18,0,0,0,0,0,15,38,0


In [67]:
df_min.columns[2:10]

Index(['cdl', 'drivers', 'laborers', 'admin', 'cashier', 'equip_op', 'supers',
       'emer_drivers'],
      dtype='object')

In [68]:
df=df_curr.copy()

def allocate(df, idx):
    '''idx is the row index == service area'''
    '''n is the minimum number of workers required to keep service area running'''
    '''adds 1 to idx when under-staffed'''
    '''starting with lowest priority service, subtract from same column and add to idx row'''
    '''if none available, adds to emergency needs'''
    for x in df_min.columns[2:10]:
        n=get_min(x, idx)
        while df[x].values[idx] < n:
            if df[x].values[idx]==n:
                break
            if df[x].values[idx+1:len(df)-2].sum()==0:
                break
            for i in range(len(df)-2, 0, -1):
                if df[x].index[i] > df[x].index[idx]:
                    for y in range(df.priority.values[i], 1,-1):
                        if df[x].index[y] > df[x].index[idx]:
                            if (df[x].values[i] > 0) & (df[x].values[idx] >= n):
                                pass
                            elif df[x].values[i]==0:
                                pass
                            elif (df[x].values[i] > 0) & (df[x].values[idx] < n):
                                while df[x].values[idx] < n:
                                    if df[x].values[i]<=0:
                                        break
                                    elif df[x].values[i] > 0 :
                                        df[x].values[i]-=1
                                        df[x].values[idx]+=1
        if df[x].values[idx:len(df)-2].sum() < n:
            if df[x].values[idx]+df.emer_drivers.values[idx] < n:
                df.emer_drivers.values[idx]+=(n-(df[x].values[idx:len(df)-2].sum()))
    return df

allocate(df, 2)
df

,svc,priority,cdl,drivers,laborers,admin,cashier,equip_op,supers,emer_drivers,min_req,today,needed
0,qrl,1,6,0,12,0,2,0,3,0,12,23,0
1,payroll,1,0,0,0,10,0,0,0,0,9,10,0
2,routine,2,72,0,148,4,0,0,16,0,207,206,1
3,nwts,2,3,0,20,0,1,5,20,0,13,49,0
4,office,3,0,0,0,14,0,0,0,0,2,18,0
5,conv_ctr,3,0,0,0,0,0,0,0,0,14,0,14
6,ds_da,3,9,6,33,0,0,9,2,0,41,59,0
7,mss,3,5,0,0,0,0,0,0,0,0,15,0
8,prop,4,0,0,33,0,0,10,0,0,0,53,0
9,spec,4,0,14,14,0,0,0,0,0,15,38,0


In [ ]:
#long form of add/subtract
df=dataframe.copy()

#qrl equip operators min=5
while df.equip_op.values[0] < 5:
    if df.equip_op.values[0]==5:
        break
    df.equip_op.values[0]+=1
    if df.equip_op.values[10] > 0:
        df.equip_op.values[10]-=1
    else:
        df.emer_drivers.values[0]+=1
        
#routine svc cdls min=72?
while df.cdl.values[2] < 72:
    if df.cdl.values[2]==72:
        break
    df.cdl.values[2]+=1
    if df.cdl.values[10] > 0:
        df.cdl.values[10]-=1
    elif df.cdl.values[9] > 0:
        df.cdl.values[9]-=1
    elif df.cdl.values[8] > 0:
        df.cdl.values[8]-=1
    elif df.cdl.values[7] > 0:
        df.cdl.values[7]-=1
    elif df.cdl.values[6] > 0:
        df.cdl.values[6]-=1
    elif df.cdl.values[5] > 0:
        df.cdl.values[5]-=1
    elif df.cdl.values[4] > 0:
        df.cdl.values[4]-=1
    elif df.cdl.values[3] > 0:
        df.cdl.values[3]-=1
    else:
        df.emer_drivers.values[2]+=1

#nwts 6 drivers
while df.cdl.values[3] < 6:
    if df.cdl.values[3]==6:
        break
    df.cdl.values[3]+=1
    if df.cdl.values[9] > 0:
        df.cdl.values[9]-=1
    elif df.cdl.values[8] > 0:
        df.cdl.values[8]-=1
    elif df.cdl.values[7] > 0:
        df.cdl.values[7]-=1
    elif df.cdl.values[6] > 0:
        df.cdl.values[6]-=1
    elif df.cdl.values[5] > 0:
        df.cdl.values[5]-=1
    elif df.cdl.values[4] > 0:
        df.cdl.values[4]-=1
    else:
        df.emer_drivers.values[2]+=1

df